In [37]:

# load and display an image with Matplotlib
from matplotlib import image
from matplotlib import pyplot
import numpy as np
# load image as pixel array
data = image.imread('./Files/ShapeModels/peppers.jpg')
# summarize shape of the pixel array
# print(data.dtype)
# print(data.shape)
# display the array of pixels as an image
#pyplot.imshow(data)
#pyplot.show()

y = np.linspace(0, len(data)-1, len(data))
x = np.linspace(0, len(data[0])-1, len(data[0]))
xv, yv= np.meshgrid(x,y) # generates correct x value if sampled at [i][j]
z = data

input_vec = [] # N x 2
output_vec = [] # N x 1
for i in range(len(x)):
    for j in range(len(y)):
        input_vec.append([x[i], y[j]])
        output_vec.append([z[i][j]])

input_vec = np.array(input_vec)
output_vec = np.array(output_vec)


In [42]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from Preprocessors.PreprocessorBase import PreprocessorBase
import copy

val_range = (0,1)
x_scaler = MinMaxScaler(feature_range=val_range)#(0, 1)) # If the data is transformed from r -> 1/r^2 the bounds become [0 (infinity), 1(max)] for body
y_scaler = MinMaxScaler(feature_range=val_range) # discrete range of values so min max makes sense
z_scaler = StandardScaler() # Can't have acceleration in negative radial direction

xy_train, xy_val, z_train, z_val = train_test_split(input_vec, output_vec, test_size=0.3)

xy_train_encoded = [xy_train]
xy_val_encoded = [xy_val]
z_train_encoded = [z_train]
z_val_encoded = [z_val]

xy_train_encoded[:,0] = x_scaler.fit_transform([xy_train[:,0]])[0]
xy_train_encoded[:,1] = y_scaler.fit_transform([xy_train[:,1]])[0]
xy_val_encoded[:,0] = x_scaler.transform([xy_val[:,0]])[0]
xy_val_encoded[:,1] = y_scaler.transform([xy_val[:,1]])[0]
z_train_encoded = z_scaler.fit_transform([z_train])
z_val_encoded = z_scaler.fit_transform([z_val])

ValueError: operands could not be broadcast together with shapes (1,5631) (13138,) (1,5631) 

In [46]:
np.shape([xy_val[:,0]][0])


(5631,)

In [31]:
from GravityModels import NN_Base

from tensorflow.keras.initializers import GlorotNormal
from tensorflow.keras.optimizers import SGD, Adadelta, Adam, Nadam, RMSprop
from tensorflow.keras.regularizers import l2

params = {}
params['epochs'] = 50
params['batch_size'] = 10
params['optimizer'] = Nadam
params['kernel_initializer'] = GlorotNormal
params['kernel_regularizer'] = 'l2'
params['first_unit'] = 256
params['first_neuron'] = 128
params['hidden_layers'] = 0
params['dropout'] = 0.4
params['lr'] = 0.1
params['losses'] = 'mean_squared_error'# 'mean_squared_error'

save_location = None
hist, model = NN_hyperparam(x_train, y_train, x_val, y_val, params, verbose=1, save_location=save_location)